In [ ]:
pip install optuna

In [ ]:
import pandas as pd
import numpy as np
import joblib

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


from sklearn.ensemble import RandomForestClassifier

import pandas as pd, optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/오일공모전/train_v1.csv')
test_data = pd.read_csv('/content/drive/MyDrive/오일공모전/test.csv')

In [ ]:
# train x, y 분리
x_train_data = train_data.loc[:,['COMPONENT_ARBITRARY','ANONYMOUS_1','YEAR','ANONYMOUS_2','AG','CO','CR','CU','FE','H2O','MN','MO','NI','PQINDEX','TI','V','V40','ZN']]
y_train_data = pd.DataFrame(train_data['Y_LABEL'])

In [ ]:
#test /// 데이터 나누기
test_log = test_data.loc[:,['ANONYMOUS_1','ANONYMOUS_2','AG','CO','CR','CU','FE','H2O','MN','MO','NI','PQINDEX','TI','V','V40','ZN']]
test_lab = test_data.loc[:,['COMPONENT_ARBITRARY','YEAR']]
test_log

In [ ]:
#train /// 데이터 나누기
x_log = x_train_data.loc[:,['ANONYMOUS_1','ANONYMOUS_2','AG','CO','CR','CU','FE','H2O','MN','MO','NI','PQINDEX','TI','V','V40','ZN']]
x_lab = x_train_data.loc[:,['COMPONENT_ARBITRARY','YEAR']]
x_log

In [ ]:
# x_train //// COMPONENT_ARBITRARY, YEAR, 제외 log1p
train_x1 = np.log1p(x_log)
train_x1

In [ ]:
# test //// COMPONENT_ARBITRARY, YEAR, 제외 log1p
test_1 = np.log1p(test_log)
test_1

In [ ]:
# train//// StandardScaler
standardScaler = StandardScaler()
scaled_train_x = standardScaler.fit_transform(train_x1)
print(scaled_train_x)

In [ ]:
#train /// 스케일링데이터 데이터프레임으로 만들어주기
scaled_train_x = pd.DataFrame(scaled_train_x)

In [ ]:
#train /// 스케일링한 데이터 컬럼명 변경
scaled_train_x.columns = ['ANONYMOUS_1','ANONYMOUS_2','AG','CO','CR','CU','FE','H2O','MN','MO','NI','PQINDEX','TI','V','V40','ZN']
scaled_train_x

In [ ]:
#train test //// 데이터 합치기
train_x = pd.concat([x_lab, scaled_train_x], axis = 1)
test1 = pd.concat([test_lab, test_1], axis = 1)

In [ ]:
# train, test /// COMPONENT_ARBITRARY, YEAR 라벨인코딩
le =LabelEncoder()
x_train_data_le = train_x
test = test1

In [ ]:
# train test /// COMPONENT_ARBITRARY 라벨인코딩 후 합치기
x_train_data_le["COMPONENT_ARBITRARY"] = le.fit_transform(train_x["COMPONENT_ARBITRARY"])
x_train_data_le["YEAR"] = le.fit_transform(train_x["YEAR"])

test["COMPONENT_ARBITRARY"] = le.fit_transform(test1["COMPONENT_ARBITRARY"])
test["YEAR"] = le.fit_transform(test1["YEAR"])

In [ ]:
# train, test 컬럼순서 맞추기
X_train = x_train_data_le[['COMPONENT_ARBITRARY','ANONYMOUS_1','YEAR','ANONYMOUS_2','AG','CO','CR','CU','FE','H2O','MN','MO','NI','PQINDEX','TI','V','V40','ZN']]
test = test[['COMPONENT_ARBITRARY','ANONYMOUS_1','YEAR','ANONYMOUS_2','AG','CO','CR','CU','FE','H2O','MN','MO','NI','PQINDEX','TI','V','V40','ZN']]

In [ ]:
test

In [ ]:
#RandomForest
kfold = StratifiedKFold(n_splits=3,random_state=123, shuffle=True)

def rf_objective(trial):
    params = {
        "n_estimators" : trial.suggest_int("n_estimators", 50,500),
        "criterion" : trial.suggest_categorical("criterion", ["gini","entropym"]),
        "max_depth" : trial.suggest_int("max_depth", 1, 100),
        'min_samples_split' : trial.suggest_int ('min_samples_split', 1, 150),
        "min_samples_leaf":trial.suggest_int("min_samples_leaf", 1, 60), 
        "max_features" : trial.suggest_int("max_features", 1, X_train.shape[1]),
        "max_leaf_nodes":trial.suggest_int("max_leaf_nodes", 2, 1000)
        }
    rf = RandomForestClassifier(n_jobs=-1, random_state=123, **params)
    score = cross_val_score(rf, X_train, y_train_data, cv=kfold, scoring="f1", n_jobs=-1)
    accuracy = score.mean()
    return accuracy
rf_study = optuna.create_study(direction="maximize")
rf_study.optimize(rf_objective, n_trials=500)
print(rf_study.best_trial.params)
print(rf_study.best_trial.values)
# optimization
opt_rf = RandomForestClassifier(
                                n_jobs=-1,
                                random_state=123,
                                n_estimators = rf_study.best_params["n_estimators"],
                                criterion = rf_study.best_params["criterion"],
                                max_depth = rf_study.best_params["max_depth"],
                                min_samples_split = rf_study.best_params["min_samples_split"],
                                min_samples_leaf = rf_study.best_params["min_samples_leaf"],
                                max_features = rf_study.best_params["max_features"],
                                max_leaf_nodes= rf_study.best_params["max_leaf_nodes"]
                                )
opt_rf.fit(X_train, y_train_data)

In [ ]:
opt = opt_rf.predict(test)

In [ ]:
opt = pd.DataFrame(opt)

In [ ]:
opt

In [ ]:
opt.to_csv('opt.csv')